# Homework

In [1]:
import pandas as pd
#github_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [2]:
df = df.iloc[:300]

In [3]:
df.sample(n=5)

,answer_llm,answer_orig,document,question,course
26,"Yes, it is possible that your confirmation ema...","The process is automated now, so you should re...",6ba259b1,Is it possible my confirmation email went to m...,machine-learning-zoomcamp
157,Before making changes to fix a GitHub push err...,Type the following command:\ngit config -l | g...,a2cfa1c9,What should the remote.origin.url look like be...,machine-learning-zoomcamp
141,To avoid getting a singular matrix error while...,I'm trying to invert the matrix but I got erro...,54ec0de4,How can I avoid getting a singular matrix erro...,machine-learning-zoomcamp
258,The command `sns.histplot(df['median_house_val...,We can use histogram:\nimport pandas as pd\nim...,bbc0fca3,What does the command sns.histplot(df['median_...,machine-learning-zoomcamp
218,You can find more resources about multiple lin...,In Question 7 we are asked to calculate\nThe i...,183a1c90,Where can I find more resources about multiple...,machine-learning-zoomcamp


In [4]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [5]:
df.iloc[0]

answer_llm     You can sign up for the course by visiting the...
answer_orig    Machine Learning Zoomcamp FAQ\nThe purpose of ...
document                                                0227b872
question                     Where can I sign up for the course?
course                                 machine-learning-zoomcamp
Name: 0, dtype: object

In [6]:
answer_llm = df.iloc[0].answer_llm

In [7]:
embedding_model.encode(answer_llm)[0]

-0.42244655

In [8]:
records_df = df.to_dict(orient='records')

In [9]:
from tqdm.auto import tqdm

In [ ]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

## Q2. Computing the dot product

Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the evaluations list

What's the 75% percentile of the score?

In [58]:
evaluations = []
evaluations_norm = []

for rec in tqdm(records_df):
    answer_llm = rec['answer_llm']
    answer_orig = rec['answer_orig']
    emb_llm = embedding_model.encode(answer_llm)
    emb_orig = embedding_model.encode(answer_orig)
    v_norm_llm = normalize(np.array(emb_llm))
    v_norm_orig = normalize(np.array(emb_orig))
    evaluations.append(emb_llm.dot(emb_orig))
    evaluations_norm.append(v_norm_llm.dot(v_norm_orig))

  0%|          | 0/300 [00:00<?, ?it/s]

In [60]:
df['evaluations'] = evaluations
df['evaluations'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547923
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: evaluations, dtype: float64

## Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we

Compute the norm of a vector
Divide each element by this norm
So, for vector v, it'll be v / ||v||

In numpy, this is how you do it:
```python
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Let's put it into a function and then compute dot product between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

In [59]:
df['evaluations_norm'] = evaluations_norm
df['evaluations_norm'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: evaluations_norm, dtype: float64

## Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

In [61]:
!pip install rouge -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [64]:
r = df.iloc[10]

In [65]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [67]:
scores['rouge-1']

{'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}

In [75]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [83]:
np.mean([scores['rouge-1']['f'], scores['rouge-1']['f'], scores['rouge-l']['f']])

0.43434342934343434

In [86]:
rouge_score_list = []
for rec in records_df:
    scores = rouge_scorer.get_scores(rec['answer_llm'], rec['answer_orig'])[0]
    rouge_score_list.append(scores['rouge-2'])

In [89]:
df_rouge_score = pd.DataFrame(rouge_score_list)

In [91]:
df_rouge_score['f'].mean()

0.20696501983423318